### Tutorial
A simple tutorial for using the gravity simulator library. This file also serves as documentation.

### Import libraries
First, we import some libraries. If the path is wrong, please correct it manually.

In [ ]:
import sys
from pathlib import Path

parent_path = str(Path.cwd().parent)
sys.path.append(parent_path)
print(f"Appended path: \"{parent_path}\"")

import numpy as np

from gravity_sim import GravitySimulator

grav_sim = GravitySimulator()
grav_sim.integration_mode = "c_lib"  # Default is "c_lib", but you can set it to "numpy"
print()
print(grav_sim.DEFAULT_SYSTEMS)
print(grav_sim.AVAILABLE_INTEGRATORS)

### Load pre-defined system

In [ ]:
system = grav_sim.create_system()
grav_sim.set_current_system(system)
system.load("solar_system")

### Adding new objects
Note that the default units are solar masses, AU and days

In [ ]:
x = np.array([-8.092549658731499e-02, 2.558381434460076e00, -6.695836142398572e-02])
v = np.array(
    [
        -1.017876585480054e-02,
        -5.452367109338154e-04,
        1.255870551153315e-03,
    ]
)
m = 1.30268459e-10
system.add(x, v, m, object_name="New object 1")

system.add_keplerian(
    semi_major_axis=4.2,
    eccentricity=0.1,
    inclination=0.1,
    longitude_of_ascending_node=1.0,
    argument_of_periapsis=1.0,
    true_anomaly=1.0,
    m=1.5e-9,
    primary_object_name="Sun",
    object_name="New object 2",
)

system.center_of_mass_correction()

### Save system

In [ ]:
system.save("Tutorial")

# You can also load a customized system stored in customized_systems.csv
# system.load("Tutorial")

### Plotting initial conditions

In [ ]:
system.plot_3d_system(
    labels=system.objects_names,
    legend=True,
)

### Launching simulation

In [ ]:
grav_sim.launch_simulation(
    integrator="leapfrog",
    tf=grav_sim.years_to_days(1000.0),
    dt=grav_sim.years_to_days(0.001),
    store_every_n=500,
    acceleration="pairwise",  # "pairwise" is the default value. Set it to "massless" if you have massless particles
)

In [ ]:
print(f"Data size = {grav_sim.data_size}")

### Plotting trajectories

In [ ]:
grav_sim.plot_2d_trajectory(
    colors=[
        "orange",
        "slategrey",
        "wheat",
        "skyblue",
        "red",
        "darkgoldenrod",
        "gold",
        "paleturquoise",
        "blue",
        None,
        None,
    ],
    labels=system.objects_names,
    legend=True,
)

### More plots

In [ ]:
# grav_sim.compute_energy()
grav_sim.plot_rel_energy(
    sol_time=grav_sim.days_to_years(grav_sim.sol_time), xlabel="Time (years)"
)

In [ ]:
# grav_sim.compute_angular_momentum()
grav_sim.plot_rel_angular_momentum(
    sol_time=grav_sim.days_to_years(grav_sim.sol_time), xlabel="Time (years)"
)

In [ ]:
grav_sim.plot_dt(
    sol_time=grav_sim.days_to_years(grav_sim.sol_time), xlabel="Time (years)"
)

### Animations

In [ ]:
grav_sim.animate_3d_traj_gif(
    is_maintain_fixed_dt=False,
    fps=30,
    dpi=200,
    plot_every_nth_point=5,
    colors=[
        "orange",
        "slategrey",
        "wheat",
        "skyblue",
        "red",
        "darkgoldenrod",
        "gold",
        "paleturquoise",
        "blue",
        None,
        None,
    ],
    labels=system.objects_names,
    legend=True,
    traj_len=50,
    file_name="tutorial.gif",
)

### Save and reading results

In [ ]:
grav_sim.save_results()

# grav_sim.load_results(file_path="Your_file_path.csv")

### Splitting simulations into multiple sessions
After simulation, you can call `grav_sim.simulator_to_system()` to convert the latest state to a new system.

Similarly, by calling `grav_sim.sol_state_to_system()`, you can convert the a state in the solution to a new system.

However, note that the $\text{d} t$ from adaptive step size integrators may change due to this.

In [ ]:
new_system1 = grav_sim.simulator_to_system()
new_system1.plot_3d_system()

new_system2 = grav_sim.sol_state_to_system(index=-1)
new_system2.plot_3d_system()

### Adaptive step size integrators

When using adaptive step size integrators, we provide tolerance rather than $\text{d}t$ to control the step size.
| Adaptive step size integrators | Recommended tolerance* |
|:-----------|:-------------|
| Runge–Kutta–Fehlberg 4(5) | $10^{-8}$ to $10^{-14}$ |
| Dormand–Prince method (DOPRI) 5(4) | $10^{-8}$ to $10^{-14}$ |
| Verner's method (DVERK) 6(5) | $10^{-8}$ to $10^{-14}$ |
| Runge–Kutta–Fehlberg 7(8) | $10^{-4}$ to $10^{-8}$ |
| IAS15 | $10^{-9}$ |

*For reference only

Using pyth-3-body, a highly chaotic system, we can easily see the difference between adaptive step size integrators and fixed step size integrators.

For RK4, the largest $\text{d}t$ we can use for this system is 2e-8.

In [ ]:
IAS15_system = grav_sim.create_system()
grav_sim.set_current_system(IAS15_system)
IAS15_system.load("pyth-3-body")

print("IAS15:")
grav_sim.launch_simulation(
    integrator="IAS15",
    tf=70.0,
    tolerance=1e-9,
)

grav_sim.plot_2d_trajectory()
grav_sim.plot_dt()

print("RK4:")
grav_sim.launch_simulation(
    integrator="rk4",
    tf=70.0,
    dt=1e-5,
    store_every_n=10000,
)

grav_sim.plot_2d_trajectory()
grav_sim.plot_dt()

### WHFast

When using WHFast, the order of adding objects matters. Since WHFast use Jacobi coordinate, we must first add the inner object. 

For convenience, you may also add the objects in any order, and then call `system.sort_by_distance(primary_object_name)` or `system.sort_by_distance(primary_object_index)`

In [ ]:
WHFast_system = grav_sim.create_system()
grav_sim.set_current_system(WHFast_system)
WHFast_system.load("solar_system")

# Shuffle the system for demonstration
indices = np.arange(WHFast_system.objects_count, dtype=int)
np.random.shuffle(indices)
WHFast_system.x = WHFast_system.x[indices]
WHFast_system.v = WHFast_system.v[indices]
WHFast_system.m = WHFast_system.m[indices]
WHFast_system.objects_names = list(np.array(WHFast_system.objects_names)[indices])

print(f"Shuffled: {WHFast_system.objects_names}")

WHFast_system.sort_by_distance(primary_object_name="Sun")

print(f"Sorted: {WHFast_system.objects_names}")

Simulating the solar system with WHFast

In [ ]:
grav_sim.launch_simulation(
    integrator="WHFast",
    tf=grav_sim.years_to_days(1000.0),
    dt=grav_sim.years_to_days(0.003),
    store_every_n=10,
)

grav_sim.plot_2d_trajectory(
    legend=True,
    labels=WHFast_system.objects_names,
)
grav_sim.plot_rel_energy()

You may run the following code block to see what happens when we don't sort the order by distance, but
note that the program may get stuck.

In [ ]:
"""
WHFast_system = grav_sim.create_system()
grav_sim.set_current_system(WHFast_system)
WHFast_system.load("solar_system")

# Shuffle the system for demonstration
indices = np.arange(WHFast_system.objects_count, dtype=int)
np.random.shuffle(indices)
WHFast_system.x = WHFast_system.x[indices]
WHFast_system.v = WHFast_system.v[indices]
WHFast_system.m = WHFast_system.m[indices]
WHFast_system.objects_names = np.array(WHFast_system.objects_names)[indices]

print(WHFast_system.objects_names)

grav_sim.launch_simulation(
    integrator="WHFast",
    tf=grav_sim.years_to_days(300.0),
    dt=grav_sim.years_to_days(0.003),
    store_every_n=10,
)

grav_sim.plot_2d_trajectory(
    legend=True,
    labels=WHFast_system.objects_names,
)
grav_sim.plot_rel_energy()
"""

# END
Here is the end of the tutorial. Check out other projects in the `examples` folder for more advanced use of this library.
* `solar_system_one_mil_yrs.ipynb`
* `asteroid_belt_animation.py`
* `kirkwood_gap.py`